# Title

This Notebook...

## Initialization

### Properties

In [ ]:
debugMode = True;

date = "20171120"
roadNumber = 2
roadsFileName = "../tests/data/BPS_20171120.txt"
detectionsFileName = "../tests/data/A2_20171120.txt"
outputDirectory = "../tests/data"

### Imports

In [ ]:
import datetime
import time as timeModule
import numpy
import scipy
import scipy.ndimage
import scipy.ndimage.filters
import matplotlib.pyplot
import matplotlib.patches

In [ ]:
import sys
sys.path.append("../scripts")
import bpsdetector
import detection
import road
import readfunctions
import parsefunctions
import utils

## Classes

In [ ]:
bpsDetector = bpsdetector.BPSDetector("00D00C03405B18200005")
print(bpsDetector)

In [ ]:
road = road.Road(12)
road.addBPSDetector(bpsDetector)
print(road)
road.indexDetectorSpaces()
print(road.getSpaceToSpaceIndex())

In [ ]:
detection = detection.Detection("00D00C03405B18200005", 1234, 1440, 120, 17)
print(detection)

## Roads

In [ ]:
bpsCodes = readfunctions.readCSVToBPSCodes(roadsFileName)

In [ ]:
for bpsCode in bpsCodes:
    print(bpsCode)

In [ ]:
roads = parsefunctions.parseBPSCodesToRoads(bpsCodes)

In [ ]:
for value in sorted(roads.values(), key = lambda x:len(x.getBPSDetectors())):
    print(value)

## Speed and Flows

In [ ]:
detections = readCSVToDetections(detectionsFileName)

In [ ]:
iterator = iter(detections)
for i in range(min(len(detections), 300)):
    detection = next(iterator)
    print(detection)

In [ ]:
speeds, flows, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex = parseDetectionsToSpeedsAndFlows(detections, roads[roadNumber])

In [ ]:
print(speeds.shape)
print(flows.shape)
print(minSpaceIndex)
print(maxSpaceIndex)
print(minTimeIndex)
print(maxTimeIndex)
matplotlib.pyplot.imshow(speeds, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flows, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsHighFlow, flowsHighFlow, maskHighFlow = removeLowFlowTimes(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsHighFlow.shape)
print(flowsHighFlow.shape)
print(sum(maskHighFlow) / (sum(maskHighFlow) + sum(~maskHighFlow)))
matplotlib.pyplot.imshow(speedsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsWorkingDetectors, flowsWorkingDetectors, maskWorkingDetectors = removeMissingDetectors(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsWorkingDetectors.shape)
print(flowsWorkingDetectors.shape)
print(sum(maskWorkingDetectors) / (sum(maskWorkingDetectors) + sum(~maskWorkingDetectors)))
matplotlib.pyplot.imshow(speedsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion

In [ ]:
congestions = parseSpeedFlowsToCongestions(speedsWorkingDetectors, flowsWorkingDetectors)

In [ ]:
print(congestions.shape)
matplotlib.pyplot.imshow(congestions, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestionsWithoutMissingValues = interpolateMissingValues(congestions)

In [ ]:
print(congestionsWithoutMissingValues.shape)
matplotlib.pyplot.imshow(congestionsWithoutMissingValues, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestionsSmoothed = applySmoothingFilter(congestionsWithoutMissingValues)

In [ ]:
print(congestionsSmoothed.shape)
matplotlib.pyplot.imshow(congestionsSmoothed, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion Filter

In [ ]:
testDataArray = numpy.random.rand(16,9)
print(testDataArray >= 0.9)

In [ ]:
scanForBoundaries(testDataArray, threshold = 0.1, depth = 0)

In [ ]:
testBoundariesList = recursiveScanForBoundaries(testDataArray, threshold = 0.1)

In [ ]:
testBoundariesList

In [ ]:
showDataWithBoundaries(testDataArray, testBoundariesList)
showDataWithBoundaries(testDataArray < 0.1, testBoundariesList)

In [ ]:
debugMode = False;

In [ ]:
congestionBoundariesList = recursiveScanForBoundaries(congestionsSmoothed)

In [ ]:
print(congestionBoundariesList)

In [ ]:
congestionBoundariesListFiltered = filterLargeCongestions(congestionBoundariesList)

In [ ]:
print(congestionBoundariesListFiltered)

In [ ]:
congestionBoundariesListWithMargins = addMargins(congestionBoundariesListFiltered)

In [ ]:
print(congestionBoundariesListWithMargins)

In [ ]:
showDataWithBoundaries(congestionsSmoothed, congestionBoundariesListWithMargins)
showDataWithBoundaries(congestionsSmoothed < 1, congestionBoundariesList)

In [ ]:
showDataWithBoundaries(speeds, congestionBoundariesListWithMargins)

In [ ]:
writeSpeedsAndFlowsToCSV(speeds, flows, congestionBoundariesListWithMargins, outputDirectory)

End of Notebook